# 關聯規則分析
## Association Rule Learning

關聯規則分析就是一種用來找出**購買A商品的人，通常也會購買B商品**這種規則。
通常會與其他演算法(例如協同過濾法)搭配成為推薦系統的一環。

這個章節主要會介紹**Apriori**與**Eclat**兩種關聯規則方法。

## 1.1. 說故事時間

某間賣場針對他們過去的銷售交易記錄進行分析，發現在傍晚時刻的尿布和啤酒銷售關聯性特別高。

![](images/beer.PNG)

- 要如何找出這樣的關聯性？ (How)
- 該如何解釋這樣的關聯性？ (Why)
- 能如何應用這樣的關聯性？ (What)

## 2.1. Apriori方法論

![](images/apriori_1.PNG)

Apriori有三個重要指標：

1. **支持度(support)** - 目標對象在所有樣本中出現的機率。
![](images/support.PNG)
2. **信賴度(confidence)** - A出現時B也出現的條件機率。
![](images/confidence.PNG)
3. **增益值(lift)** - 相較於隨機預測的表現，大於1表示正相關。
![](images/lift.PNG)

![](images/apriori_2.PNG)

Apriori依循以下四個步驟來找出關聯規則：

1. 設定support下限跟confidence下限。
2. 找出所有高於support下限的項目集(itemsets)。
3. 找出這些項目集中所有高於confidence下限的規則(rules)。
4. 將規則依照lift遞減排序。

## 2.2. Apriori程式撰寫

In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Data Importing
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None)
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


因為這邊使用apyori套件的apriori函數規定input必須是list而不能是data frame，所以我們必須對資料進行一些轉換：

In [4]:
# Data Preprocessing
transactions = []
for i in range(0, 7501):
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])
print(transactions[:5])

[['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil'], ['burgers', 'meatballs', 'eggs', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['chutney', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['turkey', 'avocado', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']]


In [5]:
# Training Apriori on the dataset
from apyori import apriori
rules = apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, max_length = 2)

參數說明：

- **transactions** -- transaction物件，必須是一個list大概長得像這樣子：[['A', 'B'], ['B', 'C']]
- **min_support** -- support下限(float)
- **min_confidence** -- confidence下限(float)
- **min_lift** -- lift下限(float)
- **max_length** -- 計算結果的項目集個數上限(integer)

In [6]:
# Visualising the results
results = list(rules)[:5]
for i in results:
    print(' -> '.join([j for j in i.items]), ',',
          'support:', "%.3f" %i.support, ',',
          'confidence:', "%.3f" %i.ordered_statistics[0][2], ',',
          'lift:', "%.3f" %i.ordered_statistics[0][3])

chicken -> light cream , support: 0.005 , confidence: 0.291 , lift: 4.844
mushroom cream sauce -> escalope , support: 0.006 , confidence: 0.301 , lift: 3.791
escalope -> pasta , support: 0.006 , confidence: 0.373 , lift: 4.701
fromage blanc -> honey , support: 0.003 , confidence: 0.245 , lift: 5.164
herb & pepper -> ground beef , support: 0.016 , confidence: 0.323 , lift: 3.292


## 3.1. Eclat方法論

Eclat與Apriori的差別在於Eclat只計算頻繁項目集(itemsets)而不計算規則(rules)，概念上相對簡單很多，計算效率上當然就也比較高。
頻繁項目集代表的意義是，同一個subset裡面的items有較高的機率同時出現。
舉例來說，尿布和啤酒有高機率在同一筆交易中被購買，那Eclat就會抓出{尿布, 啤酒}這個項目集，但並不會去探討是買尿布的人都會買啤酒？還是買啤酒的人會買尿布？

不同於Apriori，Eclat在計算上只會用到support。
![](images/support.PNG)

Eclat依循以下三個步驟來找出頻繁項目集：

1. 設定support下限。
2. 找出所有高於support下限的項目集(itemsets)。
3. 將項目集依照support遞減排序。

## 3.2. Eclat程式撰寫

課程中沒有提供Eclat的Python語法，因為講師覺得太簡單了沒必要浪費時間，以下是講師在課程問&答裡面的回覆：

there is an existing implementation of Eclat in Python but it is so much less simple than the one implemented in R so this was not worth doing it. Besides, since Eclat is somehow very similar to Apriori, with the difference that there is no confidence and lift parameters, you could still implement Eclat in Python by using the Apriori code and ignoring the confidence and lift parameters. But trust me this would have been a waste of time both for you and me if I had done a whole new implementation of Eclat in Python.

簡單來說，如果要做的話，還是可以用跟前面Apriori一樣那支程式，然後直接忽略confidence跟lift就好了，support高的就是我們要找的頻繁項目集。